# Pandas

In [1]:
import pandas as pd

In [2]:
?pd.read_csv

Signature:
pd.read_csv(
    filepath_or_buffer,
    sep=',',
    delimiter=None,
    header='infer',
    names=None,
    index_col=None,
    usecols=None,
    squeeze=False,
    prefix=None,
    mangle_dupe_cols=True,
    dtype=None,
    engine=None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace=False,
    skiprows=None,
    nrows=None,
    na_values=None,
    keep_default_na=True,
    na_filter=True,
    verbose=False,
    skip_blank_lines=True,
    parse_dates=False,
    infer_datetime_format=False,
    keep_date_col=False,
    date_parser=None,
    dayfirst=False,
    iterator=False,
    chunksize=None,
    compression='infer',
    thousands=None,
    decimal=b'.',
    lineterminator=None,
    quotechar='"',
    quoting=0,
    escapechar=None,
    comment=None,
    encoding=None,
    dialect=None,
    tupleize_cols=None,
    error_bad_lines=True,
    warn_bad_lines=True,
    skipfooter=0,
    doublequote=True,
    delim_whitespace=False,
    

In [4]:
! wc -l ../data/pubg/deaths/kill_match_stats_final_0.csv

 13426349 ../data/pubg/deaths/kill_match_stats_final_0.csv


In [8]:
def _ranges(chunksize, length):
    N = length // chunksize + 1
    for i in range(N):
        yield i*chunksize, (i+1)*chunksize

results = _ranges(10_000, 13426349)

In [9]:
# for start, end in (results):
#     df = pd_read_csv(path, skiprows=start, nrows=end)
#     ... doo stuff

## 1. Chunks

In [22]:
df = pd.read_csv('../data/pubg/deaths/kill_match_stats_final_0.csv', chunksize=500_000)

In [23]:
type(df)

pandas.io.parsers.TextFileReader

In [24]:
df1 = next(df)

In [25]:
df1.shape

(500000, 12)

In [26]:
df1.head(2)

,killed_by,killer_name,killer_placement,killer_position_x,killer_position_y,map,match_id,time,victim_name,victim_placement,victim_position_x,victim_position_y
0,Grenade,KrazyPortuguese,5.0,657725.10,146275.2,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,823,KrazyPortuguese,5.0,657725.10,146275.2
1,SCAR-L,nide2Bxiaojiejie,31.0,93091.37,722236.4,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,194,X3evolution,33.0,92238.68,723375.1


In [27]:
for i, chunk in enumerate(df):
    print(i, len(chunk))

0 500000
1 500000
2 500000
3 500000
4 500000
5 500000
6 500000
7 500000
8 500000
9 500000
10 500000
11 500000
12 500000
13 500000
14 500000
15 500000
16 500000
17 500000
18 500000
19 500000
20 500000
21 500000
22 500000
23 500000
24 500000
25 426348


## 2. Multiple Files

In [29]:
from glob import glob

In [30]:
glob('../data/pubg/deaths/*.csv')

['../data/pubg/deaths/kill_match_stats_final_3.csv',
 '../data/pubg/deaths/kill_match_stats_final_2.csv',
 '../data/pubg/deaths/kill_match_stats_final_0.csv',
 '../data/pubg/deaths/kill_match_stats_final_1.csv',
 '../data/pubg/deaths/kill_match_stats_final_4.csv']

## 3. Dask

[video tutorials](https://www.youtube.com/watch?v=6ixaCnR9ur0&t=364s)

In [31]:
from dask import dataframe as dd, multiprocessing

In [32]:
df = dd.read_csv('../data/pubg/deaths/*.csv')

In [33]:
df.columns

Index(['killed_by', 'killer_name', 'killer_placement', 'killer_position_x',
       'killer_position_y', 'map', 'match_id', 'time', 'victim_name',
       'victim_placement', 'victim_position_x', 'victim_position_y'],
      dtype='object')

In [34]:
df.head(2)

,killed_by,killer_name,killer_placement,killer_position_x,killer_position_y,map,match_id,time,victim_name,victim_placement,victim_position_x,victim_position_y
0,Grenade,KrazyPortuguese,5.0,657725.10,146275.2,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,823,KrazyPortuguese,5.0,657725.10,146275.2
1,SCAR-L,nide2Bxiaojiejie,31.0,93091.37,722236.4,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,194,X3evolution,33.0,92238.68,723375.1


In [37]:
df['diff'] = df['killer_placement'] - df['victim_placement']

In [39]:
kcount = df.groupby([ 'match_id', 'killer_name'])['map'].count()

In [40]:
kcount

Dask Series Structure:
npartitions=1
    int64
      ...
Name: map, dtype: int64
Dask Name: series-groupby-count-agg, 1272 tasks

In [ ]:
# result = kcount.compute(get=dask.multiprocessing.get)  # <<< ACTUAL COMPUTATION, WON't RUN

In [41]:
kcount.visualize(filename='schema.svg')